# OpenAI

In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableGenerator
from langchain_openai import ChatOpenAI
from typing import AsyncIterator, Iterator
import pandas as pd
import time

In [2]:
load_dotenv()

model = ChatOpenAI()

prompt = "Please echo 'Hello, world!'."

In [3]:
def transform(input: Iterator[str]) -> Iterator[str]:
    for chunk in input:
        yield chunk


async def atransform(input: AsyncIterator[str]) -> AsyncIterator[str]:
    async for chunk in input:
        yield chunk


chain = model | StrOutputParser() | RunnableGenerator(transform, atransform)

result = chain.invoke(prompt)
print("invoke: ", end="")
print(result)

iter = chain.stream(prompt)
print("stream: ", end="")
for chunk in iter:
    print(chunk, end="|")
print()

result = await chain.ainvoke(prompt)
print("ainvoke: ", end="")
print(result)

iter = chain.astream(prompt)
print("asteram: ", end="")
async for chunk in iter:
    print(chunk, end="|")
print()

invoke: Hello, world!
stream: |Hello|,| world|!||
ainvoke: Hello, world!
asteram: |Hello|,| world|!||


（tiktokenで算出した）トークン数を取得することもできる。

In [4]:
model.get_num_tokens(prompt)

8

モデルによって応答速度が違う。`max_tokens`によっても違いが出るらしい。

In [5]:
prompt = "Pythonで挨拶するプログラムを書いてください。"


def test(model_name: str, max_tokens: int) -> dict[str, any]:
    model = ChatOpenAI(model_name=model_name, max_tokens=max_tokens, temperature=0)
    start_time = time.time()
    msg = model.invoke(prompt)
    end_time = time.time()
    elapsed_time = end_time - start_time
    return {
        "model_name": model_name,
        "max_tokens": max_tokens,
        "actual_tokens": model.get_num_tokens_from_messages([msg]),
        "elapsed_time": elapsed_time,
    }


pd.DataFrame(
    [
        test(model_name="gpt-3.5-turbo", max_tokens=512),
        test(model_name="gpt-3.5-turbo", max_tokens=1024),
        test(model_name="gpt-3.5-turbo", max_tokens=2048),
        test(model_name="gpt-4", max_tokens=512),
        test(model_name="gpt-4", max_tokens=1024),
        test(model_name="gpt-4", max_tokens=2048),
        test(model_name="gpt-4", max_tokens=4096),
    ]
)

,model_name,max_tokens,actual_tokens,elapsed_time
0,gpt-3.5-turbo,512,45,0.972399
1,gpt-3.5-turbo,1024,45,0.897296
2,gpt-3.5-turbo,2048,45,0.787427
3,gpt-4,512,137,5.258275
4,gpt-4,1024,62,2.925074
5,gpt-4,2048,62,2.311672
6,gpt-4,4096,137,5.914425


In [6]:
pd.DataFrame(
    [
        test(model_name="gpt-4", max_tokens=1024),
        test(model_name="gpt-4", max_tokens=2048),
        test(model_name="gpt-4", max_tokens=3072),
        test(model_name="gpt-4", max_tokens=4096),
        test(model_name="gpt-4", max_tokens=5120),
        test(model_name="gpt-4", max_tokens=6144),
        test(model_name="gpt-4", max_tokens=7168),
    ]
)

,model_name,max_tokens,actual_tokens,elapsed_time
0,gpt-4,1024,170,5.697417
1,gpt-4,2048,137,5.241931
2,gpt-4,3072,62,2.260835
3,gpt-4,4096,156,4.608724
4,gpt-4,5120,62,2.021820
5,gpt-4,6144,62,2.425377
6,gpt-4,7168,137,3.561274


GPTさんの機嫌にもよるのかな、、、